In [1]:
from liptrf.models.linear_toy import Net
from liptrf.models.vit import ViT

import torch.nn as nn 
import torch 
import timm

from liptrf.models.vit import L2Attention
from liptrf.models.layers import LinearX
from liptrf.models.timm_vit import VisionTransformer as ViT

/home/versag/miniconda3/envs/liptrf/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
timm_model = timm.create_model('vit_tiny_patch16_224', pretrained=True)
model = ViT(patch_size=16, embed_dim=192, depth=12, num_heads=3)

In [3]:
model.load_state_dict(timm_model.state_dict(), strict=False)

_IncompatibleKeys(missing_keys=['blocks.0.attn.to_q.weight', 'blocks.0.attn.to_q.rand_x', 'blocks.0.attn.to_v.weight', 'blocks.0.attn.to_v.rand_x', 'blocks.0.attn.to_out.weight', 'blocks.0.attn.to_out.rand_x', 'blocks.0.mlp.fc1.rand_x', 'blocks.0.mlp.fc2.rand_x', 'blocks.1.attn.to_q.weight', 'blocks.1.attn.to_q.rand_x', 'blocks.1.attn.to_v.weight', 'blocks.1.attn.to_v.rand_x', 'blocks.1.attn.to_out.weight', 'blocks.1.attn.to_out.rand_x', 'blocks.1.mlp.fc1.rand_x', 'blocks.1.mlp.fc2.rand_x', 'blocks.2.attn.to_q.weight', 'blocks.2.attn.to_q.rand_x', 'blocks.2.attn.to_v.weight', 'blocks.2.attn.to_v.rand_x', 'blocks.2.attn.to_out.weight', 'blocks.2.attn.to_out.rand_x', 'blocks.2.mlp.fc1.rand_x', 'blocks.2.mlp.fc2.rand_x', 'blocks.3.attn.to_q.weight', 'blocks.3.attn.to_q.rand_x', 'blocks.3.attn.to_v.weight', 'blocks.3.attn.to_v.rand_x', 'blocks.3.attn.to_out.weight', 'blocks.3.attn.to_out.rand_x', 'blocks.3.mlp.fc1.rand_x', 'blocks.3.mlp.fc2.rand_x', 'blocks.4.attn.to_q.weight', 'blocks.4.a

In [4]:
la = L2Attention(192, 3)

In [5]:
la.to_q.weight = nn.Parameter(timm_model.blocks[0].attn.qkv.weight[:192, :].clone().detach())
la.to_v.weight = nn.Parameter(timm_model.blocks[0].attn.qkv.weight[192*2:, :].clone().detach())
la.to_out.weight = nn.Parameter(timm_model.blocks[0].attn.proj.weight.clone().detach())

In [6]:
timm_model.blocks[0].attn.qkv.bias.shape

torch.Size([576])

In [7]:
inp = torch.randn(2, 16, 192)
la(inp).shape

torch.Size([2, 16, 192])